In [1]:
import numpy as np

import torch
from torch.utils.data import DataLoader, random_split

from transformers import BloomTokenizerFast, BloomForCausalLM, AutoTokenizer


from datasets import load_dataset

from tqdm.auto import tqdm

In [2]:
%%time
# tokenizer = BloomTokenizerFast.from_pretrained("bigscience/bloom-560m")
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3large_based_on_gpt2", padding_side='left')
model = BloomForCausalLM.from_pretrained("bigscience/bloom-560m")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


CPU times: user 4.76 s, sys: 1.25 s, total: 6.01 s
Wall time: 9.94 s


In [3]:
tokenizer.pad_token = tokenizer.eos_token

In [4]:
def collate_fn(data):
    texts = [x['text'] for x in data]
    inputs = tokenizer(texts, padding=True, return_tensors='pt', max_length=64, truncation=True)
    inputs['labels'] = torch.where(inputs['input_ids'] == 50257, -100, inputs['input_ids'])
    return inputs


dataset = load_dataset('nthngdy/oscar-mini', 'unshuffled_deduplicated_ru', split='train')

train_length = int(0.99 * len(dataset))
val_length = len(dataset) - train_length

train_dataset, val_dataset = random_split(dataset, [train_length, val_length],
                                          generator=torch.Generator().manual_seed(42))
train_loader = DataLoader(train_dataset, collate_fn=collate_fn, batch_size=2, shuffle=True)
val_loader = DataLoader(val_dataset, collate_fn=collate_fn, batch_size=2)

Found cached dataset oscar-mini (/home/alexnikko/.cache/huggingface/datasets/nthngdy___oscar-mini/unshuffled_deduplicated_ru/1.0.0/d61b181331745a38dd31e8c6cc23d46566b96e255384c4421f2396af24a01dff)


In [5]:
next(iter(train_loader))

{'input_ids': tensor([[19572,   801,   768,  5123,   385,  1412,   289, 40854,   562,   272,
         22149,   305,   552,   329,  8654,  1954,   360,  8359,  4149,   872,
           718, 49641,   385,  8283,    18,   385,   923,   781, 37147,  1558,
          5289, 30085,  3245,   289, 28602,    16,  3229,   828, 18351, 49641,
           385,  8283,  6320,  5700,   411, 35243,  1781,  3258, 40583,    35,
          8545, 11291,     5],
        [50257, 50257, 50257, 50257, 50257, 50257, 50257,  4580,  1043,  5613,
          3929,  9874, 20562,    16,  2726,  5643, 22847,   289, 22721,  2257,
          5836,    16, 32293,   360, 44657, 14820,   282, 40349, 12893,    16,
          6159,  2009,  3594,    16,  9410,   416,  3320,    18,   365, 39859,
          1750, 13022, 11846,  1437, 37279,   294,  8481, 12867,  4676,  6657,
           289,  8786,    18]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 

In [6]:
for p in model.parameters():
    p.requires_grad = False

In [7]:
model.transformer.word_embeddings = torch.nn.Embedding(tokenizer.vocab_size + 1, 1024)
model.transformer.word_embeddings_layernorm = torch.nn.LayerNorm((1024,), eps=1e-5, elementwise_affine=True)

In [8]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [9]:
# for p in model.parameters():
#     p.requires_grad = False

    
device = 'cuda:0'

model.to(device)

model.eval()
perplexity_values = []
with torch.inference_mode():
    for batch in tqdm(val_loader):
        batch = {key: value.to(device) for key, value in batch.items()}
        outputs = model(**batch)
        perplexity = torch.exp(outputs.loss)
        perplexity_values.append(perplexity.item())

  0%|          | 0/575 [00:00<?, ?it/s]

In [20]:
np.mean(perplexity_values)

37.658425997858465

In [25]:
np.mean(perplexity_values)

45.671457590849506

In [10]:
print(f'not trained model perplexity = {np.mean(perplexity_values)}')

not trained model perplexity = inf


In [13]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

n_epochs = 20
n_steps_per_epoch = 1_000

for epoch in tqdm(range(n_epochs)):
    losses = []
    perplexity_values = []
    model.train()
    for i, batch in tqdm(enumerate(train_loader, start=1), total=n_steps_per_epoch):
        optimizer.zero_grad()
        
        batch = {key: value.to(device) for key, value in batch.items()}
        outputs = model(**batch)
        
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        
        losses.append(loss.item())
        perplexity_values.append(torch.exp(loss).item())
        
        if i == n_steps_per_epoch:
            break
    
    print()
    print(f'Train loss = {np.mean(losses)}')
    print(f'Train perplexity = {np.mean(perplexity_values)}')
    print()
    
    losses = []
    perplexity_values = []
    model.eval()
    with torch.inference_mode():
        for batch in tqdm(val_loader):
            batch = {key: value.to(device) for key, value in batch.items()}
            outputs = model(**batch)

            loss = outputs.loss

            losses.append(loss.item())
            perplexity_values.append(torch.exp(loss).item())
    
    print()
    print(f'Val loss = {np.mean(losses)}')
    print(f'Val perplexity = {np.mean(perplexity_values)}')
    print()

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]


Train loss = 19.513160165786744
Train perplexity = inf



  0%|          | 0/575 [00:00<?, ?it/s]


Val loss = 16.93542254904042
Val perplexity = 8.270905388244058e+25



...


Val loss = 9.942992993230405
Val perplexity = 1062308.1618654467



  0%|          | 0/1000 [00:00<?, ?it/s]


Train loss = 9.851526480197906
Train perplexity = 129624.94405145264



  0%|          | 0/575 [00:00<?, ?it/s]


Val loss = 9.860897156259288
Val perplexity = 61497.87249214504



In [14]:
len(train_dataset)

113689